In [ ]:
"""
PREVIOUS INSTALL 
"""

# import sys

# import tensorflow.keras
# import pandas as pd
# import sklearn as sk
# import tensorflow as tf

# print(f"Tensor Flow Version: {tf.__version__}")
# print(f"Keras Version: {tensorflow.keras.__version__}")
# print()
# print(f"Python {sys.version}")
# print(f"Pandas {pd.__version__}")
# print(f"Scikit-Learn {sk.__version__}")
# gpu = len(tf.config.list_physical_devices('GPU'))>0
# print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

test_mat = tf.constant(4, shape=(2, 3), dtype=float)
print(test_mat)

In [ ]:
import numpy as np
import tensorflow as tf

# tf.reset_default_graph()
print(tf.__version__)

In [ ]:
users = ['A', 'B', 'C', 'D']
movies = ['Star wars', "Shrek", "Incredibles", "Batman", "Her"]
features = ["Action", "Sci-fi", "Comedy", "Cartoon", "Drama"]

num_users = len(users)
num_movies = len(movies)
num_features = len(features)

In [ ]:
users_movies = [[5, 0, 6, 0, 8], 
                [0, 8, 7, 9, 0], 
                [8, 6, 0, 0, 3], 
                [7, 9, 0, 5, 2]]

movie_features = [[1, 1, 0, 0, 1], 
                  [0, 0, 1, 1, 0], 
                  [0, 0, 1, 1, 0], 
                  [1, 0, 0, 0, 1], 
                  [0, 0, 0, 0, 1]]

In [ ]:
users_movies = tf.constant(users_movies, dtype=tf.float32)
movie_features = tf.constant(movie_features, dtype=tf.float32)

In [ ]:
wgtd_feature_matrices = [tf.expand_dims(tf.transpose(users_movies)[:, i], axis=1) * movie_features for i in range(num_users)]

users_movies_features = tf.stack(wgtd_feature_matrices)

In [ ]:
print(wgtd_feature_matrices)

In [ ]:
print(users_movies_features)

In [ ]:
users_movies_features_sums = tf.reduce_sum(users_movies_features, axis=1)
users_movies_features_totals = tf.reduce_sum(users_movies_features_sums, axis=1)

In [ ]:
users_movies_features_sums

In [ ]:
print(users_movies_features_totals)

In [ ]:
users_features = tf.stack([users_movies_features_sums[i, :]/users_movies_features_totals[i] for i in range(num_users)], axis=0)

In [ ]:
users_features

In [ ]:
def find_users_top_features(user_index):
    features_index = tf.nn.top_k(users_features[user_index], num_features)[1]
    return tf.gather_nd(features, tf.expand_dims(features_index, axis=1))

In [ ]:
# tf.disable_v2_behavior()

# with tf.compat.v1.Session() as sess: 
#     sess.run(tf.compat.v1.global_variables_initializer())
#     users_topfeats = {}
#     for i in range(num_users): 
#         top_feats = sess.run(find_users_top_features(i))
#         users_topfeats[users[i]] = list[top_feats]

In [ ]:
users_ratings = [tf.map_fn(lambda x: tf.tensordot(users_features[i], x, axes=1), movie_features) for i in range(num_users)]

all_user_ratings = tf.stack(users_ratings)

In [ ]:
all_user_ratings

In [ ]:
all_user_ratings_new = tf.where(tf.equal(users_movies, tf.zeros_like(users_movies)), 
                        all_user_ratings,  
                        -np.inf*tf.ones_like(tf.cast(users_movies, tf.float32)))

all_user_ratings_new